# Topic Models with Gensim
## subsection of _Text Summarization and Topic Models_

* Topic Models with Gensim
    1. Text Representation with Feature Engineering
    2. Latent Semantic Indexing
    3. Implementing LSI Topic Models from Scratch
    4. Latent Dirichlet Allocation
    5. LDA Models with MALLET
    6. LDA Tuning: Finding the Optimal Number of Topics
    7. Interpreting Topic Model Results
    8. Predicting Topics for New Research Papers

## Text Representation with Feature Engineering

In [ ]:
import gensim

bigram = gensim.models.Phrases(norm_papers, min_count=20, 
                               threshold=20, delimiter=b'_') # higher threshold fewer phrases
bigram_model = gensim.models.phrases.Phraser(bigram)

# sample demonstration
print(bigram_model[norm_papers[0]][:50])

In [ ]:
norm_corpus_bigrams = [bigram_model[doc] for doc in norm_papers]

# create a dictionary representation of the documents
dictionary = gensim.corpora.Dictionary(norm_corpus_bigrams)
print('Sample word to number mappings:', list(dictionary.items())[:15])
print('Total Vocabulary Size', len(dictionary))

In [ ]:
# filer out words that occur less than 20 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below=20, no_above=0.6)
print('Total Vocabulary Size:', len(dictionary))

In [ ]:
# transforming corpus into bag of words vectors
bow_corpus = [dictionary.doc2bow(text) for text in norm_corpus_bigrams]
print(bow_corpus[1][:50])

In [ ]:
# viewing actual terms and their counts
print([(dictionary[idx], freq) for idx, freq in bow_corpus[1][:50]])

In [ ]:
# total papers in the corpus
print('Total number of papers:', len(bow_corpus))

## Latent Semantic Indexing

In [ ]:
%%time
TOTAL_TOPICS = 10
lsi_bow = gensim.models.LsiModel(bow_corpus, id2word=dictionary, num_topics=TOTAL_TOPICS, 
                                 onepass=True, chunksize=1740, power_iters=1000)

In [ ]:
for topic_id, topic in lsi_bow.print_topics(num_topics=10, num_words=20):
    print('Topic #'+str(topic_id+1)+':')
    print(topic)
    print()

In [ ]:
for n in range(TOTAL_TOPICS):
    print('Topic #'+str(n+1)+':')
    print('='*50)
    d1 = []
    d2 = []
    for term, wt in lsi_bow.show_topic(n, topn=20):
        if wt >= 0:
            d1.append((term, round(wt,3)))
        else:
            d2.append((term, round(wt,3)))
    print('Direction 1:', d1)
    print('-'*50)
    print('Direction 2:', d2)
    print('-'*50)
    print()

In [ ]:
# get U, S, VT matrices from topic model
term_topic = lsi_bow.projection.u
singular_values = lsi_bow.projection.s
topic_document = (gensim.matutils.corpus2dense(lsi_bow[bow_corpus], len(singular_values)).T / singular_values).T
term_topic.shape, singular_values.shape, topic_document.shape

In [ ]:
# document topic matrix for our LSI model
document_topics = pd.DataFrame(np.round(topic_document.T, 3), 
                               columns=['T'+str(i) for i in range(1, TOTAL_TOPICS+1)])
document_topics.head(5)

In [ ]:
document_numbers = [13, 250, 500]
for document_number in document_numbers:
    top_topics = list(document_topics.
                      columns[np.argsort(-np.absolute(document_topics.iloc[document_number].values))[:3]])
    print('Document #'+str(document_number)+':')
    print('Dominant Topics (top 3):', top_topics)
    print('Paper Summary:')
    print(papers[document_number][:500])
    print()

## Implementing LSI Topic Models from Scratch

In [ ]:
td_matrix = gensim.matutils.corpus2dense(corpus=bow_corpus, num_terms=len(dictionary))
print(td_matrix.shape)
td_matrix

In [ ]:
vocabulary = np.array(list(dictionary.values()))
print('Total vocabulary size:', len(vocabulary))
vocabulary

In [ ]:
from scipy.sparse.linalg import svds
u, s, vt = svds(td_matrix, k=TOTAL_TOPICS, maxiter=10000)
term_topic = u
singular_values = s
topic_document = vt
term_topic.shape, singular_values.shape, topic_document.shape

In [ ]:
tt_weights = term_topic.transpose() * singular_values[:,None]
tt_weights.shape

In [ ]:
top_terms = 20
topic_key_term_idxs = np.argsort(-np.absolute(tt_weights), axis=1)[:, :top_terms]
topic_keyterm_weights = np.array([tt_weights[row, columns]
                                 for row, columns in list(zip(np.arange(TOTAL_TOPICS), topic_key_term_idxs))])
topic_keyterms = vocabulary[topic_key_term_idxs]
topic_keyterms_weights = list(zip(topic_keyterms, topic_keyterm_weights))
for n in range(TOTAL_TOPICS):
    print('Topic #'+str(n+1)+':')
    print('='*50)
    d1 = []
    d2 = []
    terms, weights = topic_keyterms_weights[n]
    term_weights = sorted([(t,w) for t, w in zip(terms, weights)], key=lambda row: -abs(row[1]))
    for term, wt in term_weights:
        if wt >= 0:
            d1.append((term, round(wt, 3)))
        else:
            d2.append((term, round(wt, 3)))
    print('Direction 1:', d1)
    print('-'*50)
    print('Direction 2:', d2)
    print('-'*50)
    print()

In [ ]:
document_topics = pd.DataFrame(np.round(topic_document.T, 3), 
                               columns=['T'+str(i) for i in range(1, TOTAL_TOPICS+1)])
document_numbers = [13, 250, 500]

for document_number in document_numbers:
    top_topics = list(document_topics.
                      columns[np.argsort(-np.absolute(document_topics.iloc[document_number].values))[:3]])
    print('Document #'+str(document_number)+':')
    print('Dominant Topics (top 3):', top_topics)
    print('Paper Summary:')
    print(papers[document_number][:500])
    print()

## Latent Dirichlet Allocation

In [ ]:
%%time

lda_model = gensim.models.LdaModel(corpus=bow_corpus, id2word=dictionary, 
                                   chunksize=1740, alpha='auto', eta='auto', random_state=42, 
                                   iterations=500, num_topics=TOTAL_TOPICS, 
                                   passes=20, eval_every=None)

In [ ]:
# view topics in trained topic model
for topic_id, topic in lda_model.print_topics(num_topics=10, num_words=20):
    print('Topic #'+str(topic_id+1)+':')
    print(topic)
    print()

In [ ]:
# view overall mean coherence score of model
topics_coherences = lda_model.top_topics(bow_corpus, topn=20)
avg_coherence_score = np.mean([item[1] for item in topics_coherences])
print('Avg. Coherence Score:', avg_coherence_score)

In [ ]:
# output of topic models as tuples of terms and weights
topics_with_wts = [item[0] for item in topics_coherences]
print('LDA Topics with Weights')
print('='*50)
for idx, topic in enumerate(topics_with_wts):
    print('Topic #'+str(idx+1)+':')
    print([(term, round(wt,3)) for wt, term in topic])
    print()

In [ ]:
# view topics as a list of terms without weights, understand context or theme of each topic
print('LDA Topics without Weights')
print('='*50)
for idx, topic in enumerate(topics_with_wts):
    print('Topic #'+str(idx+1)+':')
    print([term for wt, term in topic])
    print()

In [ ]:
# use perplexity and coherence scores as measures to evaluate topic model
cv_coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, corpus=bow_corpus, 
                                                      texts=norm_corpus_bigrams, 
                                                      dictionary=dictionary, coherence='c_v')
avg_coherence_cv = cv_coherence_model_lda.get_coherence()
umass_coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, corpus=bow_corpus, 
                                                         texts=norm_corpus_bigrams, 
                                                         dictionary=dictionary, coherence='u_mass')
avg_coherence_umass = umass_coherence_model_lda.get_coherence()

perplexity = lda_model.log_perplexity(bow_corpus)

print('Avg. Coherence Score (Cv):', avg_coherence_cv)
print('Avg. Coherence Score (UMass):', avg_coherence_umass)
print('Model Perplexity:', perplexity)

## LDA Models with MALLET

In [ ]:
# download MALLET framework
# !wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

In [ ]:
# extract contents from archive
# !unzip -q mallet-2.0.8.zip

In [ ]:
MALLET_PATH = path_to_users + '/mallet-2.0.8/bin/mallet'
lda_mallet = gensim.models.wrappers.LdaMallet(mallet_path=MALLET_PATH, corpus=bow_corpus,
                                              num_topics=TOTAL_TOPICS, id2word=dictionary, 
                                              iterations=500, workers=16)

topics=[[(term, round(wt,3)) 
         for term, wt in lda_mallet.show_topic(n, topn=20)]
             for n in range(0, TOTAL_TOPICS)]

for idx, topic in enumerate(topics):
    print('Topic #'+str(idx+1)+':')
    print([term for term, wt in topic])
    print()

In [ ]:
# evaluate model using perplexity and coherence metrics
cv_coherence_model_lda_mallet = gensim.models.CoherenceModel(model=lda_mallet, corpus=bow_corpus, 
                                                             texts=norm_corpus_bigrams, 
                                                             dictionary=dictionary, 
                                                             coherence='c_v')
avg_coherence_cv = cv_coherence_model_lda_mallet.get_coherence()

umass_coherence_model_lda_mallet = gensim.models.CoherenceModel(model=lda_mallet, 
                                                                corpus=bow_corpus, 
                                                                texts=norm_corpus_bigrams, 
                                                                dictionary=dictionary, 
                                                                coherence='u_mass')

avg_coherence_umass = umass_coherence_model_lda_mallet.get_coherence()

# from STDOUT: <500> LL/token: -8.53533
perplexity = -8.53533
print('Avg. Coherence Score (Cv):', avg_coherence_cv)
print('Avg. Coherence Score (UMass):', avg_coherence_umass)
print('Model Perplexity:', perplexity)

## LDA Tuning: Finding the Optimal Number of Topics

In [ ]:
from tqdm import tqdm

# iterate and build several models with differing number of topics
# select one that has highest coherence score
def topic_model_coherence_generator(corpus, texts, dictionary, start_topic_count=2, 
                                    end_topic_count=10, step=1, cpus=1):
    models = []
    coherence_scores = []
    for topic_nums in tqdm(range(start_topic_count, end_topic_count+1, step)):
        mallet_lda_model = gensim.models.wrappers.LdaMallet(mallet_path=MALLET_PATH, 
                                                            corpus=corpus, num_topics=topic_nums, 
                                                            id2word=dictionary, iterations=500, 
                                                            workers=cpus)
        cv_coherence_model_mallet_lda = gensim.models.CoherenceModel(model=mallet_lda_model, 
                                                                     corpus=corpus, texts=texts, 
                                                                     dictionary=dictionary, 
                                                                     coherence='c_v')
        coherence_score = cv_coherence_model_mallet_lda.get_coherence()
        coherence_scores.append(coherence_score)
        models.append(mallet_lda_model)
    
    return models, coherence_scores

#lda_models, coherence_scores = topic_model_coherence_generator(corpus=bow_corpus, 
#                                                               texts=norm_corpus_bigrams, 
#                                                               dictionary=dictionary, 
#                                                               start_topic_count=2, 
#                                                               end_topic_count=30, step=1, 
#                                                               cpus=16)

In [ ]:
# load model and scores
import pickle
import numpy

filename = 'models/lda_models.sav'
lda_models = pickle.load(open(filename, 'rb'))

coherence_scores = np.genfromtxt('data/coherence_scores.csv', delimiter=',')

In [ ]:
coherence_df = pd.DataFrame({'Number of Topics': range(2, 31, 1), 
                             'Coherence Score': np.round(coherence_scores, 4)})
coherence_df.sort_values(by=['Coherence Score'], ascending=False).head(10)

In [ ]:
# plot graph showing number of topics per model and corresponding coherence scores
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

x_ax = range(2,31,1)
y_ax = coherence_scores
plt.figure(figsize=(12,6))
plt.plot(x_ax, y_ax,c='r')
plt.axhline(y=0.535, c='k', linestyle='--', linewidth=2)
plt.rcParams['figure.facecolor'] = 'white'
x1 = plt.xlabel('Number of Topics')
y1 = plt.ylabel('Coherence Scores')

In [ ]:
# based on graph, choose optimal number of topics as 20
# retrieve best model
best_model_idx = coherence_df[coherence_df['Number of Topics'] == 20].index[0]
best_lda_model = lda_models[best_model_idx]
best_lda_model.num_topics

In [ ]:
# view all the 20 topics generated by selected best model
topics = [[(term, round(wt, 3)) 
           for term, wt in best_lda_model.show_topic(n, topn=20)] 
              for n in range(0, best_lda_model.num_topics)]

for idx, topic in enumerate(topics):
    print('Topic #'+str(idx+1)+':')
    print([term for term, wt in topic])
    print()

In [ ]:
# build term topic dataframe
topics_df = pd.DataFrame([[term for term, wt in topic] 
                              for topic in topics], 
                         columns=['Term'+str(i) for i in range(1,21)], 
                         index=['Topic '+str(t) for t in range(1, best_lda_model.num_topics+1)]).T
topics_df

In [ ]:
# create topic term dataframe: each topic represented in a row with terms of topic
# represented as comma-separated string
pd.set_option('display.max_colwidth', -1)
topics_df = pd.DataFrame([', '.join([term for term, wt in topic]) 
                              for topic in topics], 
                         columns=['Terms per Topic'], 
                         index=['Topic'+str(t) for t in range (1, best_lda_model.num_topics+1)])
topics_df

## Interpreting Topic Model Results

In [ ]:
tm_results = best_lda_model[bow_corpus]

# get most dominant topic per research paper
corpus_topics = [sorted(topics, key=lambda record: -record[1])[0] for topics in tm_results]
corpus_topics[:5]

# construct master dataframe that holds base statistics
corpus_topic_df = pd.DataFrame()
corpus_topic_df['Document'] = range(0, len(papers))
corpus_topic_df['Dominant Topic'] = [item[0]+1 for item in corpus_topics]
corpus_topic_df['Contribution %'] = [round(item[1]*100, 2) for item in corpus_topics]
corpus_topic_df['Topic Desc'] = [topics_df.iloc[t[0]]['Terms per Topic'] for t in corpus_topics]
corpus_topic_df['Paper'] = papers

### Dominant Topics Distribution Across Corpusm

In [ ]:
pd.set_option('display.max_colwidth', 200)
topic_stats_df = corpus_topic_df.groupby('Dominant Topic').agg(
    {'Dominant Topic': {'Doc Count': np.size, '% Total Docs': np.size }})

topic_stats_df = topic_stats_df['Dominant Topic'].reset_index()
topic_stats_df['% Total Docs'] = topic_stats_df['% Total Docs'].apply(
    lambda row: round((row*100) / len(papers), 2))

topic_stats_df['Topic Desc'] = [topics_df.iloc[t]['Terms per Topic'] 
                                for t in range(len(topic_stats_df))]
topic_stats_df

### Dominant Topics in Specific Research Papers

In [ ]:
pd.set_option('display.max_colwidth', 200)
(corpus_topic_df[corpus_topic_df['Document'].isin(
    [681, 9, 392, 1622, 17, 906, 996, 503, 13, 733])])

### Relevant Research Papers per Topic Based on Dominance

In [ ]:
corpus_topic_df.groupby('Dominant Topic').apply(lambda topic_set: 
                                                (topic_set.sort_values(by=['Contribution %'], 
                                                                       ascending=False).iloc[0]))

## Predicting Topics for New Research Papers

In [ ]:
import glob
# papers manually downloaded from NIPS 16
# https://papers.nips.cc/book/advances-in-neural-information-processing-systems-29-2016

new_paper_files = glob.glob('test_data/nips16*.txt')
new_papers = []
for fn in new_paper_files:
    with open(fn, encoding='utf-8', errors='ignore', mode='r+') as f:
        data = f.read()
        new_papers.append(data)
print('Total New Papers', len(new_papers))

In [ ]:
# build text wrangling and feature engineering pipeline
def text_preprocessing_pipeline(documents, normalizer_fn, bigram_model):
    norm_docs = normalizer_fn(documents)
    norm_docs_bigrams = bigram_model[norm_docs]
    return norm_docs_bigrams

def bow_features_pipeline(tokenized_docs, dictionary):
    paper_bow_features = [dictionary.doc2bow(text) 
        for text in tokenized_docs]
    return paper_bow_features

norm_new_papers = text_preprocessing_pipeline(documents=new_papers, normalizer_fn=normalize_corpus, bigram_model=bigram_model)

norm_bow_features = bow_features_pipeline(tokenized_docs=norm_new_papers, dictionary=dictionary)

print(norm_new_papers[0][:30])

In [ ]:
print(norm_bow_features[0][:30])

In [ ]:
# build generic fuction to extract top N topics from any research paper using trained model
def get_topic_predictions(topic_model, corpus, topn=3):
    topic_predictions = topic_model[corpus]
    best_topics = [[(topic, round(wt, 3)) 
                        for topic, wt in sorted(topic_predictions[i], 
                                                key=lambda row: -row[1])[:topn]] 
                            for i in range(len(topic_predictions))]
    return best_topics
# putting the function in action
topic_preds = get_topic_predictions(topic_model=best_lda_model, corpus=norm_bow_features, topn=2)
topic_preds

In [ ]:
# review results for each paper
results_df = pd.DataFrame()
results_df['Papers'] = range(1, len(new_papers)+1)
results_df['Dominant Topics'] = [[topic_num+1 for topic_num, wt in item] for item in topic_preds]
res = results_df.set_index(['Papers'])['Dominant Topics'].apply(
    pd.Series).stack().reset_index(level=1, drop=True)
results_df = pd.DataFrame({'Dominant Topics': res.values}, index=res.index)
results_df['Contribution %'] = [topic_wt for topic_list in 
                                        [[round(wt*100, 2) 
                                              for topic_num, wt in item] 
                                                 for item in topic_preds] 
                                    for topic_wt in topic_list]

results_df['Topic Desc'] = [topics_df.iloc[t-1]['Terms per Topic'] for t in results_df['Dominant Topics'].values]
results_df['Paper Desc'] = [new_papers[i-1][:200] for i in results_df.index.values]

In [ ]:
pd.set_option('display.max_colwidth', 300)
results_df